In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

2024-08-11 21:13:42.261837: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-11 21:13:42.374979: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-11 21:13:42.488869: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 21:13:42.578638: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 21:13:42.604061: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-11 21:13:42.780078: I tensorflow/core/platform/cpu_feature_gu

In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
# Loading dataset
df = pd.read_csv('../clean_data/sarcasm_headlines_v2_clean.csv')

In [5]:
# Checking if dataset is correctly loaded
df.head()

,is_sarcastic,cleaned_headline
0,1,thirtysomething scientist unveil doomsday cloc...
1,0,dem rep totally nail why congress be fall shor...
2,0,eat your veggie deliciously different recipe
3,1,inclement weather prevent liar from get to work
4,1,mother come pretty close to use word stream co...


In [6]:
df.shape

(28619, 2)

In [7]:
# Checking max length
len(df['cleaned_headline'].max())

92

In [8]:
# Set parameters
max_len = 100
embedding_dim = 50

In [9]:
# Define X and y
X = df['cleaned_headline']
y = df['is_sarcastic']

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Tokenizing
tfid_vectorizer = TfidfVectorizer(max_df = 0.75, max_features = 7000, ngram_range=(1,2))
X_processed = pd.DataFrame(tfid_vectorizer.fit_transform(df['cleaned_headline']).toarray(),
                              columns=tfid_vectorizer.get_feature_names_out())

In [47]:
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# # Tokenization and padding
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(X)
# word_index = tokenizer.word_index

# X_tokenized = tokenizer.texts_to_sequences(X)

# X_padded = pad_sequences(X_tokenized, padding='post')

In [48]:
# # Ensure VOCAB_SIZE is correct
# vocab_size = len(word_index) + 1  # Adding 1 to include the padding index

# # Check and adjust indices
# X_processed = np.clip(X_padded, 0, VOCAB_SIZE - 1)

In [49]:
# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=.3, random_state=42, shuffle=True)

In [50]:
# Checking shape
X_train.shape

(20033, 7000)

In [51]:
# Instanciating model

model = Sequential([
    Embedding(vocab_size, embedding_dim),
    LSTM(64),
    Dense(1, activation='sigmoid')
])

In [52]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, batch_size=128, epochs=3, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

Epoch 1/3
 17/157 ━━━━━━━━━━━━━━━━━━━━ 13:35 6s/step - accuracy: 0.5229 - loss: 0.6922